


# Daten von der Stadt Zürich einlesen und bereinigen

In [1]:
!pip install --upgrade pip

In [2]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !pip install folium
# import folium 

!pip install geopy 
from geopy.geocoders import Nominatim 
from geopy.extra.rate_limiter import RateLimiter
from geopy.exc import GeocoderTimedOut

!pip install pyproj
from pyproj import Proj, transform

import warnings
warnings.filterwarnings('ignore')

# 1) Verkehrsunfaelle einlesen

In [32]:
# df_verkehrsunfaelle = pd.read_csv('https://data.stadt-zuerich.ch/dataset/sid_dav_strassenverkehrsunfallorte/download/RoadTrafficAccidentLocations.csv')
# df_verkehrsunfaelle.to_csv('./data/verkehrsunfaelle.csv', index=False)

In [2]:
# Abgespeicherte CSV einlesen
df_verkehrsunfaelle = pd.read_csv('./data/verkehrsunfaelle.csv')

In [3]:
df_verkehrsunfaelle.head()

,AccidentUID,AccidentType,AccidentType_de,AccidentType_fr,AccidentType_it,AccidentType_en,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentSeverityCategory_fr,AccidentSeverityCategory_it,AccidentSeverityCategory_en,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_de,RoadType_fr,RoadType_it,RoadType_en,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,CantonCode,MunicipalityCode,AccidentYear,AccidentMonth,AccidentMonth_de,AccidentMonth_fr,AccidentMonth_it,AccidentMonth_en,AccidentWeekDay,AccidentWeekDay_de,AccidentWeekDay_fr,AccidentWeekDay_it,AccidentWeekDay_en,AccidentHour,AccidentHour_text
0,A2D2677533867004E0430A865E337004,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as4,Unfall mit Sachschaden,accident avec dommages matériels,Incidente con danni materiali,Accident with property damage,False,False,False,rt433,Nebenstrasse,route secondaire,Strada secondaria,Minor road,2684605,1245194,ZH,261,2011,1,Januar,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,0.0,00h-01h
1,9FD6441F802C20A6E0430A865E3320A6,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,Accident with light injuries,False,True,False,rt433,Nebenstrasse,route secondaire,Strada secondaria,Minor road,2682382,1246980,ZH,261,2011,1,Januar,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,1.0,01h-02h
2,9FDA0DC4856A6094E0430A865E336094,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as4,Unfall mit Sachschaden,accident avec dommages matériels,Incidente con danni materiali,Accident with property damage,False,False,False,rt439,andere,autre,Altro,Other,2682791,1247749,ZH,261,2011,1,Januar,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,2.0,02h-03h
3,A3B66E42396E6000E0430A865E336000,at5,Überqueren der Fahrbahn,accident en traversant une route,Incidente nell'attraversare la carreggiata,Accident when crossing the lane(s),as3,Unfall mit Leichtverletzten,accident avec blessés légers,Incidente con feriti leggeri,Accident with light injuries,False,False,False,rt433,Nebenstrasse,route secondaire,Strada secondaria,Minor road,2681199,1247102,ZH,261,2011,1,Januar,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,2.0,02h-03h
4,9FDA0DBE8CCE9096E0430A865E339096,at0,Schleuder- oder Selbstunfall,dérapage ou perte de maîtrise,Incidente di sbandamento o per colpa propria,Accident with skidding or self-accident,as4,Unfall mit Sachschaden,accident avec dommages matériels,Incidente con danni materiali,Accident with property damage,False,False,False,rt433,Nebenstrasse,route secondaire,Strada secondaria,Minor road,2682479,1250690,ZH,261,2011,1,Januar,janvier,Gennaio,January,aw406,Samstag,samedi,Sabato,Saturday,3.0,03h-04h


## 1.1) Spalten löschen

In [5]:
# Alle Spalten löschen die mit fr, it oder en enden
del_columns = df_verkehrsunfaelle.columns[df_verkehrsunfaelle.columns.str.endswith(('fr','it','en'))]

In [6]:
df_verkehrsunfaelle = df_verkehrsunfaelle.drop(del_columns, axis = 1)

In [7]:
df_verkehrsunfaelle.shape

(48021, 21)

## 1.2) Verkehrsunfälle 2020

In [8]:
df_verkehrsunfaelle_2020 = df_verkehrsunfaelle.loc[df_verkehrsunfaelle['AccidentYear'] == 2020]

In [9]:
df_verkehrsunfaelle_2020.head(20)

,AccidentUID,AccidentType,AccidentType_de,AccidentSeverityCategory,AccidentSeverityCategory_de,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,RoadType_de,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,CantonCode,MunicipalityCode,AccidentYear,AccidentMonth,AccidentMonth_de,AccidentWeekDay,AccidentWeekDay_de,AccidentHour,AccidentHour_text
42730,9CCA869C93E0DA25E05329B9D80AF2FB,at0,Schleuder- oder Selbstunfall,as3,Unfall mit Leichtverletzten,False,True,False,rt433,Nebenstrasse,2683236,1249583,ZH,261,2020,1,Januar,aw403,Mittwoch,4.0,04h-05h
42731,9E7139B16D85C771E05329B9D80A926E,at8,Fussgängerunfall,as3,Unfall mit Leichtverletzten,True,False,False,rt433,Nebenstrasse,2682964,1248220,ZH,261,2020,1,Januar,aw403,Mittwoch,5.0,05h-06h
42732,9E7139B167FDC771E05329B9D80A926E,at1,Überholunfall oder Fahrstreifenwechsel,as4,Unfall mit Sachschaden,False,True,False,rt433,Nebenstrasse,2682811,1247444,ZH,261,2020,1,Januar,aw403,Mittwoch,5.0,05h-06h
42733,A235C3252D0E9903E05328B9D80AFCE5,at0,Schleuder- oder Selbstunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2679166,1248512,ZH,261,2020,1,Januar,aw403,Mittwoch,12.0,12h-13h
42734,9EC134679786BAC8E05329B9D80A310C,at7,Parkierunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2682395,1248019,ZH,261,2020,1,Januar,aw403,Mittwoch,14.0,14h-15h
42735,9C7F1DA927979216E05329B9D80A74E1,at1,Überholunfall oder Fahrstreifenwechsel,as3,Unfall mit Leichtverletzten,False,True,False,rt432,Hauptstrasse,2682015,1247207,ZH,261,2020,1,Januar,aw403,Mittwoch,22.0,22h-23h
42736,9C02A002EF26FE3BE05329B9D80A6395,at0,Schleuder- oder Selbstunfall,as3,Unfall mit Leichtverletzten,False,True,False,rt432,Hauptstrasse,2681515,1247181,ZH,261,2020,1,Januar,aw404,Donnerstag,7.0,07h-08h
42737,9C8C67726EDA9254E05329B9D80A777F,at9,Tierunfall,as4,Unfall mit Sachschaden,False,False,False,rt433,Nebenstrasse,2685345,1247919,ZH,261,2020,1,Januar,aw404,Donnerstag,9.0,09h-10h
42738,9D58365A52C81EDEE05329B9D80A4ADF,at1,Überholunfall oder Fahrstreifenwechsel,as4,Unfall mit Sachschaden,False,False,False,rt432,Hauptstrasse,2683852,1247189,ZH,261,2020,1,Januar,aw404,Donnerstag,17.0,17h-18h
42739,9CA385F5DA5F800DE05329B9D80AA594,at9,Tierunfall,as4,Unfall mit Sachschaden,False,False,False,rt430,Autobahn,2682388,1242752,ZH,261,2020,1,Januar,aw404,Donnerstag,17.0,17h-18h


## 1.3) Check Nulls

In [10]:
# Check Nulls
df_verkehrsunfaelle_2020.isnull().values.any()
# Keine NULLs vorhanden

False

In [11]:
df_verkehrsunfaelle_2020.shape
# 5291 - Verkehrsunfälle im 2020

(5291, 21)

## 1.4) Geodaten transformieren - LV95 -> WGS84

In [12]:
# LV95 zu WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = df_verkehrsunfaelle_2020['AccidentLocation_CHLV95_E'], df_verkehrsunfaelle_2020['AccidentLocation_CHLV95_N']
x2,y2 = transform(inProj, outProj, x1,y1)

In [13]:
df_verkehrsunfaelle_2020['AccidentLocation_WGS84_E'] = x2
df_verkehrsunfaelle_2020['AccidentLocation_WGS84_N'] = y2

In [14]:
df_verkehrsunfaelle_2020[['AccidentLocation_CHLV95_E','AccidentLocation_CHLV95_N','AccidentLocation_WGS84_E','AccidentLocation_WGS84_N']].head()

,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N
42730,2683236,1249583,47.391814,8.541109
42731,2682964,1248220,47.379590,8.537254
42732,2682811,1247444,47.372631,8.535085
42733,2679166,1248512,47.382684,8.487018
42734,2682395,1248019,47.377854,8.529684


In [15]:
# Datensatz zwischenspeichern
df_verkehrsunfaelle_2020.to_csv('./data/verkehrsunfaelle_2020.csv', index=False)

## 1.5) Reverse Geocoding - Quartier & Stadtkreis

In [ ]:
# mail von Maurizio, 24.11.21, workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

In [14]:
def get_kreis(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")
        location = locator.reverse(coordinates)
        return location.address.split(",")[-6].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [15]:
def get_quartier(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")
        location = locator.reverse(coordinates)
        return location.address.split(",")[-7].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [ ]:
kreis = []
quartier = []
for index, row in df_verkehrsunfaelle_2020.iterrows():
    coordinates = [row["AccidentLocation_WGS84_E"],row["AccidentLocation_WGS84_N"]]
    kreis.append(get_kreis(coordinates))
    quartier.append(get_quartier(coordinates))

In [ ]:
df_verkehrsunfaelle_2020["Kreis"] = kreis
df_verkehrsunfaelle_2020["Quartier"] = quartier

In [ ]:
# Einige der Kreise wurden nicht gefunden und muss mit Hilfe der Lookup Tabelle bereinigt werden
kreis_lookup = {
    'Wiedikon': 'Kreis 3',
    'Schwamendingen': 'Kreis 12',
    'Riesbach': 'Kreis 8',
    'Altstadt': 'Kreis 1',
    'Aussersihl': 'Kreis 4',
    'Industriequartier': 'Kreis 5',
    'Hochschulen': 'Kreis 1',
    'Escher Wyss': 'Kreis 5',
    'Seebach': 'Kreis 11',
    'Pfaffhausen': 'Kreis 7'
}

In [ ]:
df_verkehrsunfaelle_2020 = df_verkehrsunfaelle_2020.replace({'Kreis': kreis_lookup})

## 1.6) Jahreszeiten kategorisieren

In [50]:
jahreszeit_lookup = {
    1: 'Winter',
    2: 'Winter',
    3: 'Frühling',
    4: 'Frühling',
    5: 'Frühling',
    6: 'Sommer',
    7: 'Sommer',
    8: 'Sommer',
    9: 'Herbst',
    10: 'Herbst',
    11: 'Herbst',
    12: 'Winter'
}

In [51]:
df_verkehrsunfaelle_2020['Jahreszeit'] = df_verkehrsunfaelle_2020['AccidentMonth'].map(jahreszeit_lookup)

## 1.7) Tageszeit kategorisieren

In [56]:
df_verkehrsunfaelle_2020['ZeitKategorie'] = pd.cut(df_verkehrsunfaelle_2020['AccidentHour'], 
                    bins=[0,5,9,12,15,19,24], 
                    include_lowest=True, 
                    labels=['Nacht','Rushhour_Morgen','Morgen','Nachmittag','Rushhour_Abend','Abend'])

## 1.6) Datensatz abspeichern

In [59]:
df_verkehrsunfaelle_2020.to_csv('./data/verkehrsunfaelle_2020_mit_quartier_kreis.csv', index=False)

# 2) Verkehrszaehlung einlesen

In [189]:
# df_verkehrszaehlung_2020 = pd.read_csv('https://data.stadt-zuerich.ch/dataset/sid_dav_verkehrszaehlung_miv_od2031/download/sid_dav_verkehrszaehlung_miv_OD2031_2020.csv')

In [4]:
# Abgespeicherte CSV einlesen
df_verkehrszaehlung_2020 = pd.read_csv('./data/verkehrszaehlung_2020.csv')

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (5,14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df_verkehrszaehlung_2020.shape

(1598688, 21)

## 2.1) MessungDatZeit in DateTime umwandeln und aufsplitten

In [18]:
df_verkehrszaehlung_2020['MessungDatZeit'].dtypes

dtype('O')

In [19]:
df_verkehrszaehlung_2020['MessungDatZeit'] = pd.to_datetime(df_verkehrszaehlung_2020['MessungDatZeit'])

In [20]:
df_verkehrszaehlung_2020['MessungDatum'] = pd.to_datetime(df_verkehrszaehlung_2020['MessungDatZeit']).dt.date
df_verkehrszaehlung_2020['MessungZeit'] = pd.to_datetime(df_verkehrszaehlung_2020['MessungDatZeit']).dt.time

In [21]:
df_verkehrszaehlung_2020.head()

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungDatum,MessungZeit
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,247.0,Gemessen,2020-01-01,00:00:00
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 01:00:00,2021-02-04,364.0,Gemessen,2020-01-01,01:00:00
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 02:00:00,2021-02-04,159.0,Gemessen,2020-01-01,02:00:00
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 03:00:00,2021-02-04,108.0,Gemessen,2020-01-01,03:00:00
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 04:00:00,2021-02-04,87.0,Gemessen,2020-01-01,04:00:00


## 2.2) Check Nulls und umwandeln

In [6]:
df_verkehrszaehlung_2020.isnull().values.any()
# Es sind Nulls vorhanden im Datensatz

True

In [10]:
# AnzFahrezeige -> NaN --> 0
df_verkehrszaehlung_2020['AnzFahrzeuge'] = df_verkehrszaehlung_2020['AnzFahrzeuge'].fillna(0)

In [12]:
df_verkehrszaehlung_2020.shape

(1598688, 21)

In [11]:
df_verkehrszaehlung_2020['AnzFahrzeuge'].isnull().sum().sum()

0

In [27]:
df_verkehrszaehlung_2020.isnull().values.any()

False

## 2.3) Tag, Monat, Jahr aus dem Datum extrahieren

In [28]:
df_verkehrszaehlung_2020['Jahr'] = pd.DatetimeIndex(df_verkehrszaehlung_2020['MessungDatum']).year
df_verkehrszaehlung_2020['Monat'] = pd.DatetimeIndex(df_verkehrszaehlung_2020['MessungDatum']).month
df_verkehrszaehlung_2020['Tag'] = pd.DatetimeIndex(df_verkehrszaehlung_2020['MessungDatum']).dayofweek

In [29]:
wochentag_lookup = {
    0: 'Montag',
    1: 'Dienstag',
    2: 'Mittwoch',
    3: 'Donnerstag',
    4: 'Freitag',
    5: 'Samstag',
    6: 'Sonntag'
}

In [30]:
monat_lookup = {
    1: 'Januar',
    2: 'Februar',
    3: 'März',
    4: 'April',
    5: 'Mai',
    6: 'Juni',
    7: 'Juli',
    8: 'August',
    9: 'September',
    10: 'Oktober',
    11: 'November',
    12: 'Dezember'
}

In [31]:
df_verkehrszaehlung_2020['Wochentag'] = df_verkehrszaehlung_2020['Tag'].map(wochentag_lookup)
df_verkehrszaehlung_2020['MonatText'] = df_verkehrszaehlung_2020['Monat'].map(monat_lookup)

In [32]:
df_verkehrszaehlung_2020.head()

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungDatum,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,247.0,Gemessen,2020-01-01,00:00:00,2020,1,2,Mittwoch,Januar
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 01:00:00,2021-02-04,364.0,Gemessen,2020-01-01,01:00:00,2020,1,2,Mittwoch,Januar
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 02:00:00,2021-02-04,159.0,Gemessen,2020-01-01,02:00:00,2020,1,2,Mittwoch,Januar
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 03:00:00,2021-02-04,108.0,Gemessen,2020-01-01,03:00:00,2020,1,2,Mittwoch,Januar
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 04:00:00,2021-02-04,87.0,Gemessen,2020-01-01,04:00:00,2020,1,2,Mittwoch,Januar


## 2.4) Jahreszeit kategorisieren

In [33]:
jahreszeit_lookup = {
    1: 'Winter',
    2: 'Winter',
    3: 'Frühling',
    4: 'Frühling',
    5: 'Frühling',
    6: 'Sommer',
    7: 'Sommer',
    8: 'Sommer',
    9: 'Herbst',
    10: 'Herbst',
    11: 'Herbst',
    12: 'Winter'
}

In [34]:
df_verkehrszaehlung_2020['Jahreszeit'] = df_verkehrszaehlung_2020['Monat'].map(jahreszeit_lookup)

In [35]:
df_verkehrszaehlung_2020.head()

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungDatum,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText,Jahreszeit
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,247.0,Gemessen,2020-01-01,00:00:00,2020,1,2,Mittwoch,Januar,Winter
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 01:00:00,2021-02-04,364.0,Gemessen,2020-01-01,01:00:00,2020,1,2,Mittwoch,Januar,Winter
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 02:00:00,2021-02-04,159.0,Gemessen,2020-01-01,02:00:00,2020,1,2,Mittwoch,Januar,Winter
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 03:00:00,2021-02-04,108.0,Gemessen,2020-01-01,03:00:00,2020,1,2,Mittwoch,Januar,Winter
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 04:00:00,2021-02-04,87.0,Gemessen,2020-01-01,04:00:00,2020,1,2,Mittwoch,Januar,Winter


## 2.5) Tageszeit kategorisieren

In [36]:
hours = pd.to_datetime(df_verkehrszaehlung_2020['MessungZeit'], format='%H:%M:%S').dt.hour

df_verkehrszaehlung_2020['ZeitKategorie'] = pd.cut(hours, 
                    bins=[0,5,9,12,15,19,24], 
                    include_lowest=True, 
                    labels=['Nacht','Rushhour_Morgen','Morgen','Nachmittag','Rushhour_Abend','Abend'])

In [37]:
df_verkehrszaehlung_2020.head()

,MSID,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungDatum,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText,Jahreszeit,ZeitKategorie
0,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,247.0,Gemessen,2020-01-01,00:00:00,2020,1,2,Mittwoch,Januar,Winter,Nacht
1,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 01:00:00,2021-02-04,364.0,Gemessen,2020-01-01,01:00:00,2020,1,2,Mittwoch,Januar,Winter,Nacht
2,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 02:00:00,2021-02-04,159.0,Gemessen,2020-01-01,02:00:00,2020,1,2,Mittwoch,Januar,Winter,Nacht
3,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 03:00:00,2021-02-04,108.0,Gemessen,2020-01-01,03:00:00,2020,1,2,Mittwoch,Januar,Winter,Nacht
4,Z001M001,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 04:00:00,2021-02-04,87.0,Gemessen,2020-01-01,04:00:00,2020,1,2,Mittwoch,Januar,Winter,Nacht


In [38]:
df_temp = df_verkehrszaehlung_2020.groupby(by=["MSID","MessungDatum","ZeitKategorie"]).sum()

In [39]:
df_verkehrszaehlung_gruppiert = df_verkehrszaehlung_2020.groupby(by=["MSID","MessungDatum","ZeitKategorie"]).first()

In [40]:
df_verkehrszaehlung_gruppiert.head()

MSName  ZSID  \
MSID     MessungDatum ZeitKategorie                      
Z001M001 2020-01-01   Nacht            Unbekannt  Z001   
                      Rushhour_Morgen  Unbekannt  Z001   
                      Morgen           Unbekannt  Z001   
                      Nachmittag       Unbekannt  Z001   
                      Rushhour_Abend   Unbekannt  Z001   

                                                                   ZSName  \
MSID     MessungDatum ZeitKategorie                                         
Z001M001 2020-01-01   Nacht            Seestrasse (Strandbad Wollishofen)   
                      Rushhour_Morgen  Seestrasse (Strandbad Wollishofen)   
                      Morgen           Seestrasse (Strandbad Wollishofen)   
                      Nachmittag       Seestrasse (Strandbad Wollishofen)   
                      Rushhour_Abend   Seestrasse (Strandbad Wollishofen)   

                                            Achse  HNr      Hoehe      EKoord  \
MSID     MessungDatum ZeitKategorie                                             
Z001M001 2020-01-01   Nacht            Seestrasse  451  Unbekannt  2683009.89   
                      Rushhour_Morgen  Seestrasse  451  Unbekannt  2683009.89   
                      Morgen           Seestrasse  451  Unbekannt  2683009.89   
                      Nachmittag       Seestrasse  451  Unbekannt  2683009.89   
                      Rushhour_Abend   Seestrasse  451  Unbekannt  2683009.89   

                                          NKoord  Richtung  Knummer  \
MSID     MessungDatum ZeitKategorie                                   
Z001M001 2020-01-01   Nacht            1243936.2  auswärts      789   
                      Rushhour_Morgen  1243936.2  auswärts      789   
                      Morgen           1243936.2  auswärts      789   
                      Nachmittag       1243936.2  auswärts      789   
                      Rushhour_Abend   1243936.2  auswärts      789   

                                                        Kname  AnzDetektoren  \
MSID     MessungDatum ZeitKategorie                                            
Z001M001 2020-01-01   Nacht            Badanstalt Wollishofen              1   
                      Rushhour_Morgen  Badanstalt Wollishofen              1   
                      Morgen           Badanstalt Wollishofen              1   
                      Nachmittag       Badanstalt Wollishofen              1   
                      Rushhour_Abend   Badanstalt Wollishofen              1   

                                       D1ID       D2ID       D3ID       D4ID  \
MSID     MessungDatum ZeitKategorie                                            
Z001M001 2020-01-01   Nacht               2  Unbekannt  Unbekannt  Unbekannt   
                      Rushhour_Morgen     2  Unbekannt  Unbekannt  Unbekannt   
                      Morgen              2  Unbekannt  Unbekannt  Unbekannt   
                      Nachmittag          2  Unbekannt  Unbekannt  Unbekannt   
                      Rushhour_Abend      2  Unbekannt  Unbekannt  Unbekannt   

                                           MessungDatZeit   LieferDat  \
MSID     MessungDatum ZeitKategorie                                     
Z001M001 2020-01-01   Nacht           2020-01-01 00:00:00  2021-02-04   
                      Rushhour_Morgen 2020-01-01 06:00:00  2021-02-04   
                      Morgen          2020-01-01 10:00:00  2021-02-04   
                      Nachmittag      2020-01-01 13:00:00  2021-02-04   
                      Rushhour_Abend  2020-01-01 16:00:00  2021-02-04   

                                       AnzFahrzeuge AnzFahrzeugeStatus  \
MSID     MessungDatum ZeitKategorie                                      
Z001M001 2020-01-01   Nacht                   247.0           Gemessen   
                      Rushhour_Morgen          41.0           Gemessen   
                      Morgen                   69.0           Gemessen   
                      Nac

In [41]:
df_verkehrszaehlung_gruppiert['AnzFahrzeuge'] = df_temp['AnzFahrzeuge']

In [42]:
df_verkehrszaehlung_gruppiert.head()

MSName  ZSID  \
MSID     MessungDatum ZeitKategorie                      
Z001M001 2020-01-01   Nacht            Unbekannt  Z001   
                      Rushhour_Morgen  Unbekannt  Z001   
                      Morgen           Unbekannt  Z001   
                      Nachmittag       Unbekannt  Z001   
                      Rushhour_Abend   Unbekannt  Z001   

                                                                   ZSName  \
MSID     MessungDatum ZeitKategorie                                         
Z001M001 2020-01-01   Nacht            Seestrasse (Strandbad Wollishofen)   
                      Rushhour_Morgen  Seestrasse (Strandbad Wollishofen)   
                      Morgen           Seestrasse (Strandbad Wollishofen)   
                      Nachmittag       Seestrasse (Strandbad Wollishofen)   
                      Rushhour_Abend   Seestrasse (Strandbad Wollishofen)   

                                            Achse  HNr      Hoehe      EKoord  \
MSID     MessungDatum ZeitKategorie                                             
Z001M001 2020-01-01   Nacht            Seestrasse  451  Unbekannt  2683009.89   
                      Rushhour_Morgen  Seestrasse  451  Unbekannt  2683009.89   
                      Morgen           Seestrasse  451  Unbekannt  2683009.89   
                      Nachmittag       Seestrasse  451  Unbekannt  2683009.89   
                      Rushhour_Abend   Seestrasse  451  Unbekannt  2683009.89   

                                          NKoord  Richtung  Knummer  \
MSID     MessungDatum ZeitKategorie                                   
Z001M001 2020-01-01   Nacht            1243936.2  auswärts      789   
                      Rushhour_Morgen  1243936.2  auswärts      789   
                      Morgen           1243936.2  auswärts      789   
                      Nachmittag       1243936.2  auswärts      789   
                      Rushhour_Abend   1243936.2  auswärts      789   

                                                        Kname  AnzDetektoren  \
MSID     MessungDatum ZeitKategorie                                            
Z001M001 2020-01-01   Nacht            Badanstalt Wollishofen              1   
                      Rushhour_Morgen  Badanstalt Wollishofen              1   
                      Morgen           Badanstalt Wollishofen              1   
                      Nachmittag       Badanstalt Wollishofen              1   
                      Rushhour_Abend   Badanstalt Wollishofen              1   

                                       D1ID       D2ID       D3ID       D4ID  \
MSID     MessungDatum ZeitKategorie                                            
Z001M001 2020-01-01   Nacht               2  Unbekannt  Unbekannt  Unbekannt   
                      Rushhour_Morgen     2  Unbekannt  Unbekannt  Unbekannt   
                      Morgen              2  Unbekannt  Unbekannt  Unbekannt   
                      Nachmittag          2  Unbekannt  Unbekannt  Unbekannt   
                      Rushhour_Abend      2  Unbekannt  Unbekannt  Unbekannt   

                                           MessungDatZeit   LieferDat  \
MSID     MessungDatum ZeitKategorie                                     
Z001M001 2020-01-01   Nacht           2020-01-01 00:00:00  2021-02-04   
                      Rushhour_Morgen 2020-01-01 06:00:00  2021-02-04   
                      Morgen          2020-01-01 10:00:00  2021-02-04   
                      Nachmittag      2020-01-01 13:00:00  2021-02-04   
                      Rushhour_Abend  2020-01-01 16:00:00  2021-02-04   

                                       AnzFahrzeuge AnzFahrzeugeStatus  \
MSID     MessungDatum ZeitKategorie                                      
Z001M001 2020-01-01   Nacht                  1020.0           Gemessen   
                      Rushhour_Morgen         168.0           Gemessen   
                      Morgen                  314.0           Gemessen   
                      Nac

## 2.6) Geodaten transformieren

In [44]:
# LV95 zu WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = df_verkehrszaehlung_gruppiert['EKoord'], df_verkehrszaehlung_gruppiert['NKoord']
x2,y2 = transform(inProj, outProj, x1,y1)

In [45]:
df_verkehrszaehlung_gruppiert['Koord_WGS84_E'] = x2
df_verkehrszaehlung_gruppiert['Koord_WGS84_N'] = y2

In [220]:
# Datensatz zwischenspeichern
# df_verkehrszaehlung_gruppiert.to_csv('./data/verkehrszaehlung_2020_cleansed.csv')

## 2.7) Reverse Geocoding -Quartier & Stadtkreis

In [46]:
df_temp = df_verkehrszaehlung_gruppiert.groupby(by=["ZSID"]).first()

In [47]:
df_temp.head()

,MSName,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText,Jahreszeit,Koord_WGS84_E,Koord_WGS84_N
ZSID,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Z001,Unbekannt,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.20,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01,2021-02-04,1020.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068
Z002,Unbekannt,Albisstrasse (Widmerstrasse),Albisstrasse,152,Unbekannt,2682480.51,1243162.50,auswärts,679,Albis-/Paradies-/Widmerstr (Schulweg),1,13,Unbekannt,Unbekannt,Unbekannt,2020-01-01,2021-02-04,985.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.334167,8.529921
Z003,Unbekannt,Sood-/Leimbachstrasse,Soodstrasse,Unbekannt,Leimbachstrasse,2681573.79,1242969.91,auswärts,669,Leimbach-/Soodstr,1,16,Unbekannt,Unbekannt,Unbekannt,2020-01-01,2021-02-04,490.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.332547,8.517891
Z004,Unbekannt,Birmensdorferstrasse (Triemli),Birmensdorferstrasse,Unbekannt,90m ab Knoten,2679762.52,1246863.95,auswärts,780,Birmensdorfer-/Triemlistr,1,1,Unbekannt,Unbekannt,Unbekannt,2020-01-01,2021-02-04,602.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.367790,8.494623
Z005,Unbekannt,Albisriederstrasse (Lyrenweg),Albisriederstrasse,Unbekannt,Lyrenweg,2678874.03,1247598.55,auswärts,785,Albisriederstr/Lyrenweg,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01,2021-02-04,287.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.374504,8.482991


In [48]:
kreis = []
quartier = []
for index, row in df_temp.iterrows():
    if int(index[-2:]) % 10 == 0:
        print("am Arbeiten... aktuelle Zählstelle:", index)      # einfach um zu wissen ob er noch läuft... (mega langsam!!)

    coordinates = [row["Koord_WGS84_E"],row["Koord_WGS84_N"]]
    kreis.append(get_kreis(coordinates))
    quartier.append(get_quartier(coordinates))
print("alle Zählstellen fertig bearbeitet")

am Arbeiten... aktuelle Zählstelle: Z010
am Arbeiten... aktuelle Zählstelle: Z020
am Arbeiten... aktuelle Zählstelle: Z030
am Arbeiten... aktuelle Zählstelle: Z040
am Arbeiten... aktuelle Zählstelle: Z050
am Arbeiten... aktuelle Zählstelle: Z060
am Arbeiten... aktuelle Zählstelle: Z070
am Arbeiten... aktuelle Zählstelle: Z080
am Arbeiten... aktuelle Zählstelle: Z090
alle Zählstellen fertig bearbeitet


In [49]:
df_temp["Kreis"] = kreis
df_temp["Quartier"] = quartier

In [50]:
#  df_temp['Quartier'].value_counts().to_frame()
#  sehen alle gut aus! Keine Nachkorrektur nötig

In [51]:
# Einige der Kreise wurden nicht gefunden und muss mit Hilfe der Lookup Tabelle bereinigt werden
kreis_lookup = {
    'Wiedikon': 'Kreis 3',
    'Schwamendingen': 'Kreis 12',
    'Riesbach': 'Kreis 8',
    'Altstadt': 'Kreis 1',
    'Aussersihl': 'Kreis 4',
    'Industriequartier': 'Kreis 5'
}

In [52]:
df_temp = df_temp.replace({'Kreis': kreis_lookup})

In [53]:
df_kreis = df_temp['Kreis']
df_quartier = df_temp['Quartier']

In [54]:
df_verkehrszaehlung_gruppiert = df_verkehrszaehlung_gruppiert.reset_index()

In [55]:
df_verkehrszaehlung_komplett = df_verkehrszaehlung_gruppiert.merge(df_kreis, left_on='ZSID', right_on='ZSID')

In [56]:
df_verkehrszaehlung_komplett = df_verkehrszaehlung_komplett.merge(df_quartier, left_on='ZSID', right_on='ZSID')

In [57]:
df_verkehrszaehlung_komplett.shape

(399672, 34)

In [58]:
df_verkehrszaehlung_komplett.head()

,MSID,MessungDatum,ZeitKategorie,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText,Jahreszeit,Koord_WGS84_E,Koord_WGS84_N,Kreis,Quartier
0,Z001M001,2020-01-01,Nacht,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,1020.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen
1,Z001M001,2020-01-01,Rushhour_Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 06:00:00,2021-02-04,168.0,Gemessen,06:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen
2,Z001M001,2020-01-01,Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 10:00:00,2021-02-04,314.0,Gemessen,10:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen
3,Z001M001,2020-01-01,Nachmittag,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 13:00:00,2021-02-04,586.0,Gemessen,13:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen
4,Z001M001,2020-01-01,Rushhour_Abend,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 16:00:00,2021-02-04,805.0,Gemessen,16:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen


## 2.8) Verkehrsaufkommen kategorisieren

In [63]:
# Verkehrs in Gering, Mittel, Hoch Kategorisieren
df_verkehrszaehlung_komplett['Verkehrsaufkommen'] = df_verkehrszaehlung_komplett.groupby('MSID')[['AnzFahrzeuge']].transform(lambda x: pd.cut(x, bins = 3, labels=['Gering','Mittel','Hoch']).astype(str))

In [66]:
df_verkehrszaehlung_komplett.head(100)

,MSID,MessungDatum,ZeitKategorie,MSName,ZSID,ZSName,Achse,HNr,Hoehe,EKoord,NKoord,Richtung,Knummer,Kname,AnzDetektoren,D1ID,D2ID,D3ID,D4ID,MessungDatZeit,LieferDat,AnzFahrzeuge,AnzFahrzeugeStatus,MessungZeit,Jahr,Monat,Tag,Wochentag,MonatText,Jahreszeit,Koord_WGS84_E,Koord_WGS84_N,Kreis,Quartier,Verkehrsaufkommen
0,Z001M001,2020-01-01,Nacht,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 00:00:00,2021-02-04,1020.0,Gemessen,00:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
1,Z001M001,2020-01-01,Rushhour_Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 06:00:00,2021-02-04,168.0,Gemessen,06:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
2,Z001M001,2020-01-01,Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 10:00:00,2021-02-04,314.0,Gemessen,10:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
3,Z001M001,2020-01-01,Nachmittag,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 13:00:00,2021-02-04,586.0,Gemessen,13:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
4,Z001M001,2020-01-01,Rushhour_Abend,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 16:00:00,2021-02-04,805.0,Gemessen,16:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
5,Z001M001,2020-01-01,Abend,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-01 20:00:00,2021-02-04,439.0,Gemessen,20:00:00,2020,1,2,Mittwoch,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
6,Z001M001,2020-01-02,Nacht,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-02 00:00:00,2021-02-04,125.0,Gemessen,00:00:00,2020,1,3,Donnerstag,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
7,Z001M001,2020-01-02,Rushhour_Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-02 06:00:00,2021-02-04,243.0,Gemessen,06:00:00,2020,1,3,Donnerstag,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
8,Z001M001,2020-01-02,Morgen,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-02 10:00:00,2021-02-04,485.0,Gemessen,10:00:00,2020,1,3,Donnerstag,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering
9,Z001M001,2020-01-02,Nachmittag,Unbekannt,Z001,Seestrasse (Strandbad Wollishofen),Seestrasse,451,Unbekannt,2683009.89,1243936.2,auswärts,789,Badanstalt Wollishofen,1,2,Unbekannt,Unbekannt,Unbekannt,2020-01-02 13:00:00,2021-02-04,714.0,Gemessen,13:00:00,2020,1,3,Donnerstag,Januar,Winter,47.341059,8.537068,Kreis 2,Wollishofen,Gering


## 2.9) Datensatz abspeichern

In [67]:
df_verkehrszaehlung_komplett.to_csv('./data/verkehrszaehlung_2020_mit_quartier_kreis.csv')